# 01 — Data Audit

**Tweede Kamer Open Data · Speech-to-Vote Project**

Profile every processed parquet file: row counts, column types, null rates, date ranges.
Documents the dataset for the team and validates the data chain for speech-vote linking.

In [ ]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path("../data/processed")
parquet_files = sorted(DATA_DIR.glob("*.parquet"))
print(f"Found {len(parquet_files)} parquet files")

In [ ]:
def profile_table(path: Path) -> dict:
    df = pd.read_parquet(path)
    name = path.stem
    
    # Date columns for range detection
    date_cols = [c for c in df.columns if c in ('Datum', 'Geboortedatum', 'GewijzigdOp', 'Van', 'TotEnMet', 'PeriodeVan', 'PeriodeTotEnMet')]
    date_range = None
    if date_cols:
        for col in date_cols:
            vals = pd.to_datetime(df[col], errors='coerce')
            vals = vals.dropna()
            if len(vals) > 0:
                date_range = f"{vals.min().strftime('%Y-%m-%d')} to {vals.max().strftime('%Y-%m-%d')}"
                break
    
    # Null rate per column
    null_pct = (df.isnull().sum() / len(df) * 100).round(1)
    high_null = null_pct[null_pct > 50].to_dict()
    
    return {
        "entity": name,
        "rows": len(df),
        "columns": len(df.columns),
        "columns_list": list(df.columns),
        "dtypes": df.dtypes.astype(str).to_dict(),
        "null_rate": null_pct.to_dict(),
        "date_range": date_range,
        "memory_mb": df.memory_usage(deep=True).sum() / 1024 / 1024,
    }

profiles = []
for p in parquet_files:
    try:
        profiles.append(profile_table(p))
    except Exception as e:
        print(f"Error: {p.name}: {e}")

audit_df = pd.DataFrame([{k: v for k, v in p.items() if k not in ('columns_list', 'dtypes', 'null_rate')} for p in profiles])
audit_df = audit_df.sort_values('rows', ascending=False)
audit_df

## Summary by entity

In [ ]:
audit_df[['entity', 'rows', 'columns', 'date_range', 'memory_mb']].to_string(index=False)

## Critical data chain: Speech → Vote

**Vergadering** → **Verslag** (XML transcript) → **Activiteit** → **Agendapunt** → **Besluit** → **Stemming**

Verify these links exist:

In [ ]:
def check_chain():
    activiteit = pd.read_parquet(DATA_DIR / "Activiteit.parquet")
    agendapunt = pd.read_parquet(DATA_DIR / "Agendapunt.parquet")
    besluit = pd.read_parquet(DATA_DIR / "Besluit.parquet")
    stemming = pd.read_parquet(DATA_DIR / "Stemming.parquet")
    
    # Agendapunt has Activiteit_Id
    ap_act = agendapunt[agendapunt['Activiteit_Id'].notna()]
    matched_act = set(ap_act['Activiteit_Id']) & set(activiteit['Id'])
    print(f"Agendapunt -> Activiteit: {len(matched_act):,} matched activiteiten")
    
    # Besluit has Agendapunt_Id
    besluit_voted = besluit[besluit['Id'].isin(stemming['Besluit_Id'].dropna())]
    print(f"Besluiten with votes: {len(besluit_voted):,}")
    print(f"Total Stemming records: {len(stemming):,}")
    print(f"Vote distribution: {stemming['Soort'].value_counts().to_dict()}")
    
    # Activiteit -> Agendapunt -> Besluit chain
    ap_ids = set(agendapunt['Id'])
    besluit_ap = besluit_voted[besluit_voted['Agendapunt_Id'].isin(ap_ids)]
    act_ids_from_ap = set(agendapunt[agendapunt['Id'].isin(besluit_ap['Agendapunt_Id'])]['Activiteit_Id'].dropna())
    print(f"Activiteiten with voted Besluiten: {len(act_ids_from_ap):,}")

check_chain()

## Key entities (Stemming, Besluit, Fractie, Persoon)

In [ ]:
stemming = pd.read_parquet(DATA_DIR / "Stemming.parquet")
print("Stemming columns:", stemming.columns.tolist())
print("\nSoort (vote type):", stemming['Soort'].value_counts())
print("\nTop parties (ActorFractie):", stemming['ActorFractie'].value_counts().head(10))

In [ ]:
besluit = pd.read_parquet(DATA_DIR / "Besluit.parquet")
print("BesluitSoort (decision types):", besluit['BesluitSoort'].value_counts().head(15))

In [ ]:
fractie = pd.read_parquet(DATA_DIR / "Fractie.parquet")
active = fractie[fractie['DatumInactief'].isna()]
print(f"Active factions: {len(active)}")
print(active[['Afkorting', 'NaamNL', 'AantalZetels']].head(15).to_string(index=False))

In [ ]:
vergadering = pd.read_parquet(DATA_DIR / "Vergadering.parquet")
verslag_path = DATA_DIR / "Verslag.parquet"
verslag = pd.read_parquet(verslag_path) if verslag_path.exists() else None
print(f"Vergadering: {len(vergadering):,} rows")
if verslag is not None:
    print(f"Verslag (metadata): {len(verslag):,} rows")
else:
    print("Verslag: use raw Verslag.json or fetch_verslagen for XMLs")